In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd /content/gdrive/MyDrive/

/content/gdrive/MyDrive


In [ ]:
pwd

'/content/gdrive/MyDrive'

In [1]:
cd /content/gdrive/MyDrive/UNO_MLAI/UNO-main

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/UNO_MLAI/UNO-main'
/content


In [ ]:
class UNO(nn.Module):
    def __init__(self,in_width, width,pad = 8, factor = 3/4):
        super(UNO, self).__init__()
        
        self.in_width = in_width # input channel
        self.width = width 
        self.factor = factor
        self.padding = pad 
        
        self.fc = nn.Linear(self.in_width, self.width//2)

        self.fc0 = nn.Linear(self.width//2, self.width) # input channel is 3: (a(x, y), x, y)
        
        self.conv0 = OperatorBlock_2D(self.width, 2*factor*self.width,64, 64, 28, 28)

        self.conv1 = OperatorBlock_2D(2*factor*self.width, 4*factor*self.width, 32, 32, 16,16)

        self.conv2 = OperatorBlock_2D(4*factor*self.width, 8*factor*self.width, 16, 16,8,8)
        
        self.conv3 = OperatorBlock_2D(8*factor*self.width, 8*factor*self.width, 16, 16,8,8)
        
        self.conv4 = OperatorBlock_2D(8*factor*self.width, 4*factor*self.width, 32, 32,8,8)

        self.conv5 = OperatorBlock_2D(8*factor*self.width, 2*factor*self.width, 64, 64,16,16)

        self.conv6 = OperatorBlock_2D(4*factor*self.width, self.width, 85, 85,28,28) # will be reshaped

        self.fc1 = nn.Linear(1*self.width, 2*self.width)
        self.fc2 = nn.Linear(2*self.width, 1)

    def forward(self, x):
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)


        x_fc = self.fc(x)
        x_fc = F.gelu(x_fc)

        x_fc0 = self.fc0(x_fc)
        x_fc0 = F.gelu(x_fc0)
        
        x_fc0 = x_fc0.permute(0, 3, 1, 2)
        
        
        x_fc0 = F.pad(x_fc0, [0,self.padding, 0,self.padding])#
        
        D1,D2 = x_fc0.shape[-2],x_fc0.shape[-1]
        

        x_c0 = self.conv0(x_fc0,int(D1*self.factor),int(D2*self.factor))

        x_c1 = self.conv1(x_c0 ,D1//2,D2//2)

        x_c2 = self.conv2(x_c1 ,D1//4,D2//4)
    
        x_c3 = self.conv3(x_c2,D1//4,D2//4) 

        x_c4 = self.conv4(x_c3,D1//2,D2//2)
        x_c4 = torch.cat([x_c4, x_c1], dim=1)

        x_c5 = self.conv5(x_c4,int(D1*self.factor),int(D2*self.factor))
        x_c5 = torch.cat([x_c5, x_c0], dim=1)

        x_c6 = self.conv6(x_c5,D1,D2)

        
        if self.padding!=0:
            x_c6 = x_c6[..., 0:-self.padding, 0:-self.padding]

        x_c6 = x_c6.permute(0, 2, 3, 1)
        
        x_fc1 = self.fc1(x_c6)
        x_fc1 = F.gelu(x_fc1)
        
        x_out = self.fc2(x_fc1)
        
        return x_out
    
    def get_grid(self, shape, device):
        batchsize, size_x, size_y = shape[0], shape[1], shape[2]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)


NameError: ignored

In [ ]:
def FDM_Darcy(u, a, D=1):
    batchsize = u.size(0)
    size = u.size(1)
    u = u.reshape(batchsize, size, size)
    a = a.reshape(batchsize, size, size)
    dx = D / (size - 1)
    dy = dx

    # ux: (batch, size-2, size-2)
    ux = (u[:, 2:, 1:-1] - u[:, :-2, 1:-1]) / (2 * dx)
    uy = (u[:, 1:-1, 2:] - u[:, 1:-1, :-2]) / (2 * dy)

    a = a[:, 1:-1, 1:-1]

    aux = a * ux
    auy = a * uy
    auxx = (aux[:, 2:, 1:-1] - aux[:, :-2, 1:-1]) / (2 * dx)
    auyy = (auy[:, 1:-1, 2:] - auy[:, 1:-1, :-2]) / (2 * dy)
    Du = - (auxx + auyy)
    return Du


def darcy_loss(u, a):
    batchsize = u.size(0)
    size = u.size(1)
    u = u.reshape(batchsize, size, size)
    a = a.reshape(batchsize, size, size)
    lploss = LpLoss(size_average=True)

    # index_x = torch.cat([torch.tensor(range(0, size)), (size - 1) * torch.ones(size), torch.tensor(range(size-1, 1, -1)),
    #                      torch.zeros(size)], dim=0).long()
    # index_y = torch.cat([(size - 1) * torch.ones(size), torch.tensor(range(size-1, 1, -1)), torch.zeros(size),
    #                      torch.tensor(range(0, size))], dim=0).long()

    # boundary_u = u[:, index_x, index_y]
    # truth_u = torch.zeros(boundary_u.shape, device=u.device)
    # loss_u = lploss.abs(boundary_u, truth_u)

    Du = FDM_Darcy(u, a)
    f = torch.ones(Du.shape, device=u.device)
    loss_f = lploss.rel(Du, f)

    # im = (Du-f)[0].detach().cpu().numpy()
    # plt.imshow(im)
    # plt.show()

    # loss_f = FDM_Darcy(u, a)
    # loss_f = torch.mean(loss_f)
    return loss_f


def train(args, config):
    seed = random.randint(1, 10000)
    print(f'Random seed :{seed}')
    torch.manual_seed(seed)
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    data_config = config['data']
    dataset = DarcyFlow(data_config['datapath'],
                        nx=data_config['nx'], sub=data_config['sub'],
                        offset=data_config['offset'], num=data_config['n_sample'])
    dataloader = DataLoader(dataset, batch_size=config['train']['batchsize'])
    model = FNO2d(modes1=config['model']['modes1'],
                  modes2=config['model']['modes2'],
                  fc_dim=config['model']['fc_dim'],
                  layers=config['model']['layers'],
                  act=config['model']['act']).to(device)
    # Load from checkpoint
    if 'ckpt' in config['train']:
        ckpt_path = config['train']['ckpt']
        ckpt = torch.load(ckpt_path)
        model.load_state_dict(ckpt['model'])
        print('Weights loaded from %s' % ckpt_path)
    optimizer = Adam(model.parameters(), betas=(0.9, 0.999),
                         lr=config['train']['base_lr'])
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,
                                                     milestones=config['train']['milestones'],
                                                     gamma=config['train']['scheduler_gamma'])
    train_2d_operator(model,
                      dataloader,
                      optimizer, scheduler,
                      config, rank=0, log=args.log,
                      project=config['log']['project'],
                      group=config['log']['group'])

In [ ]:
import torch
from torchsummary import summary
model = torch.load('Darcy-D13-421.pt')

optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay,amsgrad = False)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)
best_error = 100000.0
myloss = LpLoss(size_average=False)
  for ep in range(epochs):
      model.train()
      t1 = default_timer()
      train_l2 = 0
      for x, y in train_loader:
          x, y = x.cuda(), y.cuda()
          batch_size = x.shape[0]
          optimizer.zero_grad()
          out = model(x).reshape(batch_size, s, s)

          loss = myloss(out.view(batch_size,-1), y.view(batch_size,-1))
          loss.backward()

          optimizer.step()
          train_l2 += loss.item()
          del x,y,out,loss
          gc.collect()
      torch.cuda.empty_cache()

      scheduler.step()